In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
def prepare_data(df):
    """
    Prepare the data for modeling with proper handling of mixed data types.
    """
    df = df.copy()
    
    # Define feature sets
    predictor_features = [
        'average_weather_before',
        'most_common_surface', 'most_common_roof', 'average_snaps_before',
        'sum_travel_magnitude', 'sum_tz_diff_magnitude', 'sum_elevation_difference',
        'prev_weather', 'prev_wind', 'prev_surface', 'prev_roof',
        'prev_snaps', 'prev_travel_magnitude', 'prev_is_international',
        'prev_elevation_difference', 'prev_travel_direction', 'prev_elevation_difference_abs_m'
    ]
    
    performance_features = [
        'pass_cmp_x', 'pass_att', 'pass_yds', 'pass_td', 'pass_int', 'pass_sacked_x',
        'pass_sacked_yds', 'pass_long', 'pass_rating', 'rush_att_x', 'rush_yds_x',
        'rush_td_x', 'rush_long', 'targets_x', 'rec_x', 'rec_yds_x', 'rec_td_x',
        'rec_long', 'fumbles', 'fumbles_lost', 'def_int_x', 'def_int_yds',
        'def_int_td', 'def_int_long', 'pass_defended', 'sacks_x', 'tackles_combined_x',
        'tackles_solo', 'tackles_assists', 'tackles_loss', 'qb_hits', 'fumbles_rec',
        'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'xpm', 'xpa', 'fgm',
        'fga', 'punt', 'punt_yds', 'punt_yds_per_punt', 'punt_long', 'kick_ret',
        'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td', 'kick_ret_long',
        'punt_ret', 'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td',
        'punt_ret_long', 'pass_first_down', 'pass_first_down_pct', 'pass_target_yds',
        'pass_tgt_yds_per_att', 'pass_air_yds', 'pass_air_yds_per_cmp',
        'pass_air_yds_per_att', 'pass_yac', 'pass_yac_per_cmp', 'pass_drops',
        'pass_drop_pct', 'pass_poor_throws', 'pass_poor_throw_pct', 'pass_sacked_y',
        'pass_blitzed', 'pass_hurried', 'pass_hits', 'pass_pressured',
        'pass_pressured_pct', 'rush_scrambles', 'rush_scrambles_yds_per_att',
        'rush_first_down', 'rush_yds_before_contact', 'rush_yds_bc_per_rush',
        'rush_yac', 'rush_yac_per_rush', 'rush_broken_tackles',
        'rush_broken_tackles_per_rush', 'rec_first_down', 'rec_air_yds',
        'rec_air_yds_per_rec', 'rec_yac', 'rec_yac_per_rec', 'rec_adot',
        'rec_broken_tackles', 'rec_broken_tackles_per_rec', 'rec_drops',
        'rec_drop_pct', 'rec_target_int', 'rec_pass_rating', 'def_targets',
        'def_cmp', 'def_cmp_perc', 'def_cmp_yds', 'def_yds_per_cmp',
        'def_yds_per_target', 'def_cmp_td', 'def_pass_rating', 'def_tgt_yds_per_att',
        'def_air_yds', 'def_yac', 'blitzes', 'qb_hurry', 'qb_knockdown', 'pressures',
        'tackles_missed', 'tackles_missed_pct', 'avg_snap_count_after'
    ]
    
    # Filter to only existing columns
    predictor_features = [col for col in predictor_features if col in df.columns]
    performance_features = [col for col in performance_features if col in df.columns]
    
    print(f"Found {len(predictor_features)} predictor features")
    print(f"Found {len(performance_features)} performance features")
    
    # Create binary target for injury prediction
    print("\nCreating injury severity target...")
    if 'game_status' in df.columns:
        # Clean game_status column
        df['game_status'] = df['game_status'].astype(str).str.strip().str.lower()
        
        # Map to severity
        severity_map = {
            'out': 2,
            'doubtful': 1,
            'questionable': 1,
            'probable': 0,
            'nan': np.nan,
            'none': np.nan
        }
        
        df['injury_severity'] = df['game_status'].map(severity_map)
        print(f"Injury severity distribution:")
        print(df['injury_severity'].value_counts(dropna=False))
    else:
        print("Warning: 'game_status' column not found, cannot create injury severity")
        df['injury_severity'] = np.nan
    
    # Handle missing values separately for numeric and categorical columns
    print("\nHandling missing values...")
    
    # Identify numeric vs categorical columns
    numeric_predictors = []
    categorical_predictors = []
    
    for col in predictor_features:
        if col in df.columns:
            # Try to convert to numeric to check
            numeric_test = pd.to_numeric(df[col], errors='coerce')
            numeric_count = numeric_test.notna().sum()
            total_count = df[col].notna().sum()
            
            if numeric_count > 0 and (numeric_count / total_count > 0.5 if total_count > 0 else False):
                numeric_predictors.append(col)
            else:
                categorical_predictors.append(col)
    
    print(f"Numeric predictors: {len(numeric_predictors)}")
    print(f"Categorical predictors: {len(categorical_predictors)}")
    
    # Fill missing values for numeric columns
    if numeric_predictors:
        for col in numeric_predictors:
            # Extract numbers from strings like "56%", "88%", etc.
            if df[col].dtype == object:
                # Clean the column first
                df[col] = df[col].astype(str).str.extract(r'([-+]?\d*\.?\d+)')[0]
            
            # Convert to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Fill missing with median
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
    
    # Fill missing values for categorical columns
    if categorical_predictors:
        for col in categorical_predictors:
            # Fill with mode (most common value)
            if df[col].notna().sum() > 0:
                mode_val = df[col].mode().iloc[0] if len(df[col].mode()) > 0 else 'Unknown'
            else:
                mode_val = 'Unknown'
            
            df[col] = df[col].fillna(mode_val)
    
    # Fill missing values for performance features
    for col in performance_features:
        if col in df.columns:
            # Clean if string
            if df[col].dtype == object:
                df[col] = df[col].astype(str).str.extract(r'([-+]?\d*\.?\d+)')[0]
            
            # Convert to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Fill missing with median
            if df[col].notna().sum() > 0:
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
    
    # Encode categorical features
    print("\nEncoding categorical features...")
    
    # Identify which categorical columns to encode
    categorical_cols = ['most_common_surface', 'most_common_roof', 
                       'prev_surface', 'prev_roof', 'prev_travel_direction']
    
    # Filter to only columns that exist and are categorical
    categorical_cols = [col for col in categorical_cols if col in df.columns and col in categorical_predictors]
    
    label_encoders = {}
    for col in categorical_cols:
        try:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le
            print(f"  Encoded {col}: {len(le.classes_)} categories")
        except Exception as e:
            print(f"  Warning: Could not encode {col}: {e}")
            # Keep as is or create dummy variables
    
    print(f"\nFinal prepared data shape: {df.shape}")
    
    return df, predictor_features, performance_features

In [3]:
def model_injury_risk(df, predictor_features):
    """
    Model 1: Predict injury risk based on pre-injury features.
    """
    print("=" * 60)
    print("MODEL 1: INJURY RISK PREDICTION")
    print("=" * 60)
    
    # Check if we have the target column
    if 'injury_severity' not in df.columns:
        print("Warning: 'injury_severity' column not found")
        return {}
    
    # Check for NaN in target
    nan_count = df['injury_severity'].isna().sum()
    print(f"Target column 'injury_severity' has {nan_count} NaN values")
    
    # Remove rows with NaN in target
    df_clean = df.dropna(subset=['injury_severity']).copy()
    
    if len(df_clean) == 0:
        print("Error: No valid injury severity data after removing NaN")
        return {}
    
    print(f"Using {len(df_clean)} samples after removing NaN from target")
    
    # Prepare data
    X = df_clean[predictor_features]
    y = df_clean['injury_severity']  # Multi-class: 0=minor, 1=moderate, 2=severe
    
    # Convert y to integer if it's float
    if y.dtype == float:
        y = y.astype(int)
    
    print(f"Target distribution:")
    print(y.value_counts().sort_index())
    
    # Check if we have enough samples for each class
    class_counts = y.value_counts()
    min_samples = class_counts.min()
    print(f"Minimum class samples: {min_samples}")
    
    if min_samples < 5:
        print("Warning: Some classes have very few samples, consider collapsing categories")
    
    # Split data
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
    except ValueError as e:
        print(f"Stratified split failed: {e}")
        print("Using regular split instead...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
    
    print(f"\nTrain shape: {X_train.shape}, Test shape: {X_test.shape}")

    non_numeric = X_train.select_dtypes(include=["object"])
    print(non_numeric.columns)
    print(non_numeric.head())
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train models
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'XGBoost': xgb.XGBClassifier(random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        
        try:
            # Train
            model.fit(X_train_scaled, y_train)
            
            # Predict
            y_pred = model.predict(X_test_scaled)
            
            # Evaluate
            print(f"Accuracy: {model.score(X_test_scaled, y_test):.3f}")
            print("Classification Report:")
            print(classification_report(y_test, y_pred, 
                  target_names=['Minor', 'Moderate', 'Severe']))
            
            # Feature importance
            if hasattr(model, 'feature_importances_'):
                importances = model.feature_importances_
                feature_importance = pd.DataFrame({
                    'feature': predictor_features,
                    'importance': importances
                }).sort_values('importance', ascending=False)
                
                print("\nTop 10 Most Important Features for Injury Prediction:")
                print(feature_importance.head(10).to_string(index=False))
                
                # Plot feature importance
                plt.figure(figsize=(10, 6))
                plt.barh(feature_importance.head(10)['feature'], 
                        feature_importance.head(10)['importance'])
                plt.xlabel('Importance')
                plt.title(f'{name} - Top 10 Injury Risk Features')
                plt.gca().invert_yaxis()
                plt.tight_layout()
                plt.show()
                
                results[name] = {
                    'model': model,
                    'feature_importance': feature_importance,
                    'scaler': scaler
                }
                
        except Exception as e:
            print(f"Error training {name}: {e}")
            continue
    
    return results

In [4]:

# ============================================================================
# MODEL 2: Predict Post-Injury Performance
# ============================================================================

def model_post_injury_performance(df, predictor_features, performance_features):
    """
    Model 2: Predict post-injury performance based on injury characteristics.
    """
    print("\n" + "=" * 60)
    print("MODEL 2: POST-INJURY PERFORMANCE PREDICTION")
    print("=" * 60)
    
    # Use injury features as predictors
    injury_features = ['injury_lcoation', 'practice_status', 'game_status', 'position']
    
    # Prepare data
    df_model = df.copy()
    
    # Encode injury features
    le_injury = LabelEncoder()
    df_model['injury_lcoation_encoded'] = le_injury.fit_transform(
        df_model['injury_lcoation'].fillna('Unknown').astype(str)
    )
    
    le_practice = LabelEncoder()
    df_model['practice_status_encoded'] = le_practice.fit_transform(
        df_model['practice_status'].fillna('Unknown').astype(str)
    )
    
    le_position = LabelEncoder()
    df_model['position_encoded'] = le_position.fit_transform(
        df_model['position'].fillna('Unknown').astype(str)
    )
    
    # Combine features
    X_features = ['injury_lcoation_encoded', 'practice_status_encoded', 'position_encoded']
    X = df_model[X_features]
    
    performance_results = {}
    
    # Predict each performance metric
    print(f"\nPredicting {len(performance_features)} performance metrics...")
    
    for i, target_col in enumerate(performance_features[:10]):  # Limit to first 10 for demo
        if target_col not in df_model.columns:
            continue
            
        print(f"\n{target_col}...")
        
        y = df_model[target_col]
        
        # Remove rows where target is NaN
        valid_idx = y.notna()
        X_clean = X[valid_idx]
        y_clean = y[valid_idx]
        
        if len(y_clean) < 20:  # Need enough data
            continue
        
        # Split
        X_train, X_test, y_train, y_test = train_test_split(
            X_clean, y_clean, test_size=0.2, random_state=42
        )
        
        # Scale
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train_scaled, y_train)
        
        # Predict
        y_pred = model.predict(X_test_scaled)
        
        # Evaluate
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"  MSE: {mse:.3f}, R²: {r2:.3f}")
        
        performance_results[target_col] = {
            'model': model,
            'mse': mse,
            'r2': r2,
            'scaler': scaler,
            'feature_importance': dict(zip(X_features, model.feature_importances_))
        }
    
    return performance_results

In [5]:

# ============================================================================
# MODEL 3: Integrated Analysis - Connecting Injury Risk to Performance
# ============================================================================

def integrated_analysis(df, predictor_features, performance_features):
    """
    Model 3: Connect injury risk factors to post-injury performance.
    """
    print("\n" + "=" * 60)
    print("MODEL 3: INTEGRATED ANALYSIS")
    print("Connecting Injury Risk Factors to Performance Impact")
    print("=" * 60)
    
    # 3.1: Cluster players by injury risk profile
    from sklearn.cluster import KMeans
    
    X_risk = df[predictor_features].fillna(df[predictor_features].median())
    
    # Scale
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_risk)
    
    # Find optimal clusters
    inertias = []
    for k in range(2, 10):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
    
    # Elbow method visualization
    plt.figure(figsize=(10, 6))
    plt.plot(range(2, 10), inertias, 'bo-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal Clusters')
    plt.show()
    
    # Choose k (e.g., 4)
    k = 4
    kmeans = KMeans(n_clusters=k, random_state=42)
    df['risk_cluster'] = kmeans.fit_predict(X_scaled)
    
    # 3.2: Analyze performance by risk cluster
    print("\nPerformance by Risk Cluster:")
    cluster_performance = {}
    
    for cluster in range(k):
        cluster_mask = df['risk_cluster'] == cluster
        cluster_size = cluster_mask.sum()
        
        print(f"\nCluster {cluster} (n={cluster_size}):")
        
        # Get average performance metrics for this cluster
        cluster_avg = df.loc[cluster_mask, performance_features].mean()
        
        # Top 5 performance metrics where this cluster differs most from overall mean
        overall_avg = df[performance_features].mean()
        diff = (cluster_avg - overall_avg).abs()
        top_diffs = diff.nlargest(5)
        
        print("  Most affected performance metrics:")
        for metric, diff_val in top_diffs.items():
            cluster_val = cluster_avg[metric]
            overall_val = overall_avg[metric]
            pct_change = ((cluster_val - overall_val) / overall_val * 100) if overall_val != 0 else 0
            print(f"    {metric}: {cluster_val:.2f} vs {overall_val:.2f} ({pct_change:+.1f}%)")
    
    # 3.3: Key injury risk factors analysis
    print("\n" + "=" * 40)
    print("KEY FINDINGS: Injury Risk Factors")
    print("=" * 40)
    
    # Correlation analysis
    injury_correlations = {}
    for feature in predictor_features:
        corr = df[feature].corr(df['injury_severity'])
        if not pd.isna(corr):
            injury_correlations[feature] = abs(corr)
    
    top_corrs = pd.Series(injury_correlations).nlargest(10)
    print("\nTop 10 factors correlated with injury severity:")
    for feature, corr in top_corrs.items():
        direction = "increases" if df[feature].corr(df['injury_severity']) > 0 else "decreases"
        print(f"  {feature}: {corr:.3f} ({direction} risk)")
    
    # 3.4: Performance impact by injury type
    print("\n" + "=" * 40)
    print("Performance Impact by Injury Type")
    print("=" * 40)
    
    if 'injury_lcoation' in df.columns:
        injury_locations = df['injury_lcoation'].value_counts().head(5).index
        
        for location in injury_locations:
            location_mask = df['injury_lcoation'] == location
            if location_mask.sum() > 10:  # Enough samples
                print(f"\n{location}:")
                
                # Compare performance before vs average
                location_perf = df.loc[location_mask, performance_features].mean()
                overall_perf = df[performance_features].mean()
                
                # Find biggest differences
                perf_diff = (location_perf - overall_perf) / overall_perf * 100
                perf_diff = perf_diff.replace([np.inf, -np.inf], np.nan).dropna()
                
                biggest_drops = perf_diff.nsmallest(3)
                biggest_gains = perf_diff.nlargest(3)
                
                print("  Biggest performance drops:")
                for metric, pct in biggest_drops.items():
                    print(f"    {metric}: {pct:+.1f}%")
                
                print("  Biggest performance gains:")
                for metric, pct in biggest_gains.items():
                    print(f"    {metric}: {pct:+.1f}%")
    
    return df

In [6]:

# ============================================================================
# MAIN PIPELINE
# ============================================================================

def run_injury_analysis_pipeline(df):
    """
    Run the complete injury analysis pipeline.
    """
    print("INJURY ANALYSIS PIPELINE")
    print("=" * 60)
    
    # Step 1: Prepare data
    df_prepared, predictor_features, performance_features = prepare_data(df)
    print(f"Data prepared: {len(df_prepared)} samples")
    print(f"Predictor features: {len(predictor_features)}")
    print(f"Performance features: {len(performance_features)}")
    
    # Step 2: Model 1 - Injury Risk Prediction
    injury_risk_results = model_injury_risk(df_prepared, predictor_features)
    
    # Step 3: Model 2 - Post-Injury Performance Prediction
    performance_results = model_post_injury_performance(
        df_prepared, predictor_features, performance_features
    )
    
    # Step 4: Model 3 - Integrated Analysis
    df_analyzed = integrated_analysis(df_prepared, predictor_features, performance_features)
    
    # Step 5: Summary and Recommendations
    print("\n" + "=" * 60)
    print("SUMMARY & RECOMMENDATIONS")
    print("=" * 60)
    
    # Key takeaways from Model 1
    print("\n1. KEY INJURY RISK FACTORS:")
    rf_importance = injury_risk_results.get('Random Forest', {}).get('feature_importance')
    if rf_importance is not None:
        top_risks = rf_importance.head(5)
        for _, row in top_risks.iterrows():
            print(f"   • {row['feature']}: Importance = {row['importance']:.3f}")
    
    # Key takeaways from Model 2
    print("\n2. PERFORMANCE IMPACT FINDINGS:")
    if performance_results:
        # Sort by R² to find most predictable metrics
        predictable_metrics = sorted(
            [(metric, results['r2']) for metric, results in performance_results.items()],
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        print("   Most predictable post-injury metrics:")
        for metric, r2 in predictable_metrics:
            print(f"   • {metric}: R² = {r2:.3f}")
    
    # Practical recommendations
    print("\n3. PRACTICAL RECOMMENDATIONS:")
    print("   a) Monitor high-risk factors identified (travel, workload, environment)")
    print("   b) Adjust training based on injury location and severity predictions")
    print("   c) Use performance predictions to set realistic return expectations")
    print("   d) Consider player position in risk assessment")
    
    return {
        'data': df_analyzed,
        'injury_risk_models': injury_risk_results,
        'performance_models': performance_results,
        'predictor_features': predictor_features,
        'performance_features': performance_features
    }

In [7]:

# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def create_visualizations(results):
    """
    Create comprehensive visualizations for the analysis.
    """
    df = results['data']
    predictor_features = results['predictor_features']
    
    # 1. Injury Severity Distribution
    plt.figure(figsize=(10, 6))
    df['injury_severity'].value_counts().sort_index().plot(kind='bar')
    plt.xlabel('Injury Severity (0=Minor, 1=Moderate, 2=Severe)')
    plt.ylabel('Count')
    plt.title('Distribution of Injury Severity')
    plt.xticks(rotation=0)
    plt.show()
    
    # 2. Top Predictor Features Heatmap
    if len(predictor_features) >= 5:
        top_features = predictor_features[:10]  # Top 10 features
        
        plt.figure(figsize=(12, 8))
        correlation_matrix = df[top_features + ['injury_severity']].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Correlation Heatmap: Predictor Features vs Injury Severity')
        plt.tight_layout()
        plt.show()
    
    # 3. Performance Before vs After Injury (for top 3 positions)
    if 'position' in df.columns and 'avg_snap_count_after' in df.columns:
        top_positions = df['position'].value_counts().head(3).index
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        for idx, position in enumerate(top_positions):
            pos_mask = df['position'] == position
            if pos_mask.sum() > 0:
                axes[idx].scatter(
                    df.loc[pos_mask, 'average_snaps_before'],
                    df.loc[pos_mask, 'avg_snap_count_after'],
                    alpha=0.6
                )
                axes[idx].plot([0, 100], [0, 100], 'r--', alpha=0.5)  # Reference line
                axes[idx].set_xlabel('Average Snaps Before Injury')
                axes[idx].set_ylabel('Average Snaps After Injury')
                axes[idx].set_title(f'{position} - Snaps Before vs After')
                axes[idx].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

In [8]:
# ============================================================================
# EXECUTION
# ============================================================================


df = pd.read_csv('clean_data/agged_data2.csv')

df["prev_wind"] = (
    df["prev_wind"]
    .astype(str)
    .str.replace("mph", "", regex=False)
    .str.strip()
)

df["prev_wind"] = pd.to_numeric(df["prev_wind"], errors="coerce")

df["prev_travel_magnitude"] = df["prev_travel_magnitude"].replace({
    "home": 0,
    "short": 1,
    "medium": 2,
    "long": 3
})

numeric_cols = [
    "average_humidity_before",
    "average_wind_before",
    "prev_humidity",
    "prev_wind"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


# Run the complete pipeline
analysis_results = run_injury_analysis_pipeline(df)

# Create visualizations
create_visualizations(analysis_results)

# Save results
analysis_results['data'].to_csv('injury_analysis_results.csv', index=False)
print("\nAnalysis complete! Results saved to 'injury_analysis_results.csv'")

INJURY ANALYSIS PIPELINE
Found 17 predictor features
Found 111 performance features

Creating injury severity target...
Injury severity distribution:
injury_severity
NaN    280024
1.0     17132
2.0     10480
Name: count, dtype: int64

Handling missing values...
Numeric predictors: 12
Categorical predictors: 5

Encoding categorical features...
  Encoded most_common_surface: 8 categories
  Encoded most_common_roof: 4 categories
  Encoded prev_surface: 8 categories
  Encoded prev_roof: 4 categories
  Encoded prev_travel_direction: 4 categories

Final prepared data shape: (307636, 138)
Data prepared: 307636 samples
Predictor features: 17
Performance features: 111
MODEL 1: INJURY RISK PREDICTION
Target column 'injury_severity' has 280024 NaN values
Using 27612 samples after removing NaN from target
Target distribution:
injury_severity
1    17132
2    10480
Name: count, dtype: int64
Minimum class samples: 10480

Train shape: (22089, 17), Test shape: (5523, 17)
Index([], dtype='object')
Empty

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# ============================================================================
# DATA PREPARATION WITH COMPLETE ERROR HANDLING
# ============================================================================

def prepare_data_complete(df):
    """
    Complete data preparation with all error handling.
    """
    df = df.copy()
    
    # Define feature sets
    predictor_features = [
        'average_weather_before', 'average_humidity_before', 'average_wind_before',
        'most_common_surface', 'most_common_roof', 'average_snaps_before',
        'sum_travel_magnitude', 'sum_tz_diff_magnitude', 'sum_elevation_difference',
        'prev_weather', 'prev_humidity', 'prev_wind', 'prev_surface', 'prev_roof',
        'prev_snaps', 'prev_travel_magnitude', 'prev_is_international',
        'prev_elevation_difference', 'prev_travel_direction', 'prev_elevation_difference_abs_m'
    ]
    
    # Filter to only existing columns
    predictor_features = [col for col in predictor_features if col in df.columns]
    
    print(f"Found {len(predictor_features)} predictor features")
    
    # Step 1: Clean all predictor features
    print("\nStep 1: Cleaning predictor features...")
    
    for col in predictor_features:
        if col in df.columns:
            # Check data type
            if df[col].dtype == object:
                print(f"  Cleaning {col} (object type)...")
                
                # Replace common non-numeric strings
                df[col] = df[col].astype(str).str.lower()
                non_numeric_strings = ['unknown', 'n/a', 'na', 'nan', 'none', 'null', '']
                df[col] = df[col].replace(non_numeric_strings, np.nan)
                
                # Extract numbers from strings
                df[col] = df[col].str.extract(r'([-+]?\d*\.?\d+)')[0]
            
            # Convert to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Step 2: Create proper binary target
    print("\nStep 2: Creating injury target...")
    
    if 'game_status' in df.columns:
        # Clean and create binary target
        df['game_status'] = df['game_status'].astype(str).str.strip().str.lower()
        
        # Create binary target: 1 for injured, 0 for not injured
        injury_keywords = ['out', 'doubtful', 'questionable']
        df['is_injured'] = df['game_status'].isin(injury_keywords).astype(int)
        
        # Ensure we only have 0 and 1
        df['is_injured'] = df['is_injured'].clip(0, 1)
        
        print(f"Injury distribution: {df['is_injured'].value_counts().to_dict()}")
    else:
        print("Warning: No game_status column, creating dummy target")
        df['is_injured'] = 0
    
    # Step 3: Remove rows with all NaN in predictors
    print("\nStep 3: Removing invalid rows...")
    
    # Check for rows with all NaN in predictors
    nan_mask = df[predictor_features].isna().all(axis=1)
    if nan_mask.any():
        print(f"  Removing {nan_mask.sum()} rows with all NaN predictors")
        df = df[~nan_mask]
    
    # Step 4: Impute missing values
    print("\nStep 4: Imputing missing values...")
    
    # Use median for numeric columns
    for col in predictor_features:
        if col in df.columns:
            if df[col].isna().any():
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
    
    # Final check: ensure all values are finite
    for col in predictor_features:
        if col in df.columns:
            # Replace infinite values
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
            df[col] = df[col].fillna(df[col].median())
    
    print(f"\nFinal data shape: {df.shape}")
    print(f"Injured: {df['is_injured'].sum()}, Not injured: {len(df) - df['is_injured'].sum()}")
    
    return df, predictor_features

# ============================================================================
# ROBUST INJURY RISK MODEL
# ============================================================================

def robust_injury_risk_model(df, predictor_features):
    """
    Robust injury risk model with full error handling.
    """
    print("=" * 60)
    print("ROBUST INJURY RISK MODEL")
    print("=" * 60)
    
    # Prepare data
    X = df[predictor_features].copy()
    y = df['is_injured'].copy()
    
    print(f"Data shape: {X.shape}")
    print(f"Target distribution:")
    print(f"  0 (Not injured): {sum(y == 0)}")
    print(f"  1 (Injured): {sum(y == 1)}")
    
    # Check if we have enough injured samples
    if sum(y == 1) < 20:
        print(f"Warning: Only {sum(y == 1)} injured samples, model may not be reliable")
    
    # Ensure X is numeric and has no NaN
    X = X.apply(pd.to_numeric, errors='coerce')
    X = X.fillna(X.median())
    
    # Convert to numpy arrays
    X_array = X.values.astype(float)
    y_array = y.values.astype(int)
    
    # Remove any rows with NaN in X_array
    nan_mask = np.isnan(X_array).any(axis=1)
    if nan_mask.any():
        print(f"Removing {nan_mask.sum()} rows with NaN after conversion")
        X_array = X_array[~nan_mask]
        y_array = y_array[~nan_mask]
    
    print(f"\nFinal arrays shape: X={X_array.shape}, y={y_array.shape}")
    
    # Check class distribution
    unique_classes, class_counts = np.unique(y_array, return_counts=True)
    print(f"Unique classes in y: {unique_classes}")
    print(f"Class counts: {class_counts}")
    
    # We need at least 2 classes
    if len(unique_classes) < 2:
        print("Error: Only one class in target variable")
        return {}
    
    # Split data
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X_array, y_array, test_size=0.2, random_state=42, stratify=y_array
        )
    except ValueError as e:
        print(f"Stratified split failed: {e}")
        print("Using regular split...")
        X_train, X_test, y_train, y_test = train_test_split(
            X_array, y_array, test_size=0.2, random_state=42
        )
    
    print(f"\nTrain: {X_train.shape}, Test: {X_test.shape}")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train models
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        
        try:
            # Train
            model.fit(X_train_scaled, y_train)
            
            # Predict
            y_pred = model.predict(X_test_scaled)
            
            # Evaluate
            accuracy = model.score(X_test_scaled, y_test)
            
            print(f"Accuracy: {accuracy:.3f}")
            print("Classification Report:")
            
            # Handle class labels properly
            class_labels = ['Not Injured', 'Injured']
            print(classification_report(y_test, y_pred, target_names=class_labels, zero_division=0))
            
            # Feature importance
            if hasattr(model, 'feature_importances_'):
                importances = model.feature_importances_
                feature_importance = pd.DataFrame({
                    'feature': predictor_features,
                    'importance': importances
                }).sort_values('importance', ascending=False)
                
                print("\nTop 10 Most Important Features:")
                print(feature_importance.head(10).to_string(index=False))
                
                # Plot feature importance
                plt.figure(figsize=(10, 6))
                plt.barh(feature_importance.head(10)['feature'], 
                        feature_importance.head(10)['importance'])
                plt.xlabel('Importance')
                plt.title(f'{name} - Top 10 Injury Risk Features')
                plt.gca().invert_yaxis()
                plt.tight_layout()
                plt.show()
                
                results[name] = {
                    'model': model,
                    'feature_importance': feature_importance,
                    'scaler': scaler,
                    'accuracy': accuracy
                }
                
        except Exception as e:
            print(f"Error training {name}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    return results

# ============================================================================
# SIMPLE ANALYSIS FUNCTIONS
# ============================================================================

def analyze_injury_patterns(df):
    """
    Simple analysis of injury patterns.
    """
    print("\n" + "=" * 60)
    print("INJURY PATTERN ANALYSIS")
    print("=" * 60)
    
    results = {}
    
    # 1. Injury rate by position
    if 'position' in df.columns and 'is_injured' in df.columns:
        injury_by_position = df.groupby('position')['is_injured'].agg(['mean', 'count'])
        injury_by_position = injury_by_position.sort_values('mean', ascending=False)
        
        print("\n1. Injury Rate by Position:")
        for position, row in injury_by_position.head(10).iterrows():
            print(f"   {position}: {row['mean']:.1%} ({int(row['count'])} players)")
        
        results['injury_by_position'] = injury_by_position
    
    # 2. Most common injury locations
    if 'injury_lcoation' in df.columns:
        injury_locations = df['injury_lcoation'].value_counts().head(10)
        
        print("\n2. Most Common Injury Locations:")
        for location, count in injury_locations.items():
            print(f"   {location}: {count} injuries")
        
        results['common_injury_locations'] = injury_locations
    
    # 3. Correlation with numeric features
    numeric_features = ['average_snaps_before', 'sum_travel_magnitude', 
                       'sum_elevation_difference', 'average_weather_before']
    
    if 'is_injured' in df.columns:
        print("\n3. Correlation with Injury Status:")
        for feature in numeric_features:
            if feature in df.columns:
                try:
                    # Ensure numeric
                    feature_series = pd.to_numeric(df[feature], errors='coerce')
                    corr = feature_series.corr(df['is_injured'])
                    print(f"   {feature}: {corr:.3f}")
                except:
                    print(f"   {feature}: Could not calculate correlation")
    
    return results

# ============================================================================
# MAIN PIPELINE
# ============================================================================

def run_complete_analysis(df):
    """
    Complete analysis pipeline with full error handling.
    """
    print("COMPLETE INJURY ANALYSIS PIPELINE")
    print("=" * 60)
    
    try:
        # Step 1: Prepare data
        print("\n[Step 1/3] Preparing data...")
        df_prepared, predictor_features = prepare_data_complete(df)
        
        # Step 2: Run injury risk model
        print("\n[Step 2/3] Running injury risk model...")
        model_results = robust_injury_risk_model(df_prepared, predictor_features)
        
        # Step 3: Analyze patterns
        print("\n[Step 3/3] Analyzing injury patterns...")
        analysis_results = analyze_injury_patterns(df_prepared)
        
        # Summary
        print("\n" + "=" * 60)
        print("ANALYSIS COMPLETE - SUMMARY")
        print("=" * 60)
        
        if model_results:
            print("✓ Injury Risk Model: SUCCESS")
            for name, result in model_results.items():
                print(f"  - {name} Accuracy: {result['accuracy']:.3f}")
                top_feature = result['feature_importance'].iloc[0]
                print(f"  - Top predictor: {top_feature['feature']} (importance: {top_feature['importance']:.3f})")
        else:
            print("✗ Injury Risk Model: FAILED")
        
        print("✓ Pattern Analysis: COMPLETE")
        
        # Save results
        df_prepared.to_csv('injury_analysis_complete.csv', index=False)
        print("\n✓ Results saved to 'injury_analysis_complete.csv'")
        
        return {
            'data': df_prepared,
            'model_results': model_results,
            'analysis_results': analysis_results,
            'predictor_features': predictor_features
        }
        
    except Exception as e:
        print(f"\n✗ Analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# ============================================================================
# DEBUGGING AND VALIDATION
# ============================================================================

def validate_data(df):
    """
    Validate data before running analysis.
    """
    print("DATA VALIDATION CHECK")
    print("=" * 60)
    
    # Check required columns
    required_cols = ['game_status']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"✗ Missing required columns: {missing_cols}")
        return False
    
    # Check game_status values
    print("\nGame Status Distribution:")
    if 'game_status' in df.columns:
        status_counts = df['game_status'].value_counts(dropna=False)
        for status, count in status_counts.head(10).items():
            print(f"  {status}: {count}")
    
    # Check predictor features
    predictor_features = [
        'average_snaps_before', 'sum_travel_magnitude', 'average_weather_before'
    ]
    
    print("\nPredictor Feature Check:")
    for feature in predictor_features:
        if feature in df.columns:
            null_count = df[feature].isna().sum()
            dtype = df[feature].dtype
            print(f"  {feature}: dtype={dtype}, nulls={null_count}/{len(df)}")
        else:
            print(f"  {feature}: NOT FOUND")
    
    return True

# ============================================================================
# EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Load your data
    df = pd.read_csv('clean_data/agged_data2.csv')
    
    # Validate data first
    if validate_data(df):
        # Run complete analysis
        results = run_complete_analysis(df)
        
        if results:
            print("\n✓ Analysis pipeline completed successfully!")
        else:
            print("\n✗ Analysis pipeline failed")
    else:
        print("\n✗ Data validation failed, cannot proceed with analysis")

DATA VALIDATION CHECK

Game Status Distribution:
  nan: 280024
  Questionable: 15455
  Out: 10480
  Doubtful: 1677

Predictor Feature Check:
  average_snaps_before: dtype=float64, nulls=13016/307636
  sum_travel_magnitude: dtype=float64, nulls=6796/307636
  average_weather_before: dtype=float64, nulls=17024/307636
COMPLETE INJURY ANALYSIS PIPELINE

[Step 1/3] Preparing data...
Found 20 predictor features

Step 1: Cleaning predictor features...
  Cleaning most_common_surface (object type)...
  Cleaning most_common_roof (object type)...
  Cleaning prev_humidity (object type)...
  Cleaning prev_wind (object type)...
  Cleaning prev_surface (object type)...
  Cleaning prev_roof (object type)...
  Cleaning prev_travel_magnitude (object type)...
  Cleaning prev_is_international (object type)...
  Cleaning prev_travel_direction (object type)...

Step 2: Creating injury target...
Injury distribution: {0: 280024, 1: 27612}

Step 3: Removing invalid rows...
  Removing 6796 rows with all NaN pred